In [1]:
# Fuente https://github.com/googlesamples/mediapipe/blob/main/examples/hand_landmarker/python/hand_landmarker.ipynb
# Utilidades visualización para el demostrador Face landmark detection

import cv2
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import mediapipe as mp
from enum import Enum, auto
import random
import cvzone

MARGIN = 10  # pixels
FONT_SIZE = 1
FONT_THICKNESS = 1
HANDEDNESS_TEXT_COLOR = (88, 205, 54) # vibrant green


class GameMove(Enum):
    ROCK = "ROCK"
    PAPER = "PAPER"
    SCISSORS = "SCISSOR"

player = GameMove.ROCK
opponent = GameMove.ROCK


def draw_landmarks_on_image(rgb_image, detection_result):
  hand_landmarks_list = detection_result.hand_landmarks
  gestures_list = detection_result.gestures
  annotated_image = np.copy(rgb_image)

  # Loop through the detected hands to visualize.
  for idx in range(len(hand_landmarks_list)):
    hand_landmarks = hand_landmarks_list[idx]
    gestures = gestures_list[idx]

    # Draw the hand landmarks.
    hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    hand_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in hand_landmarks
    ])
    solutions.drawing_utils.draw_landmarks(
      annotated_image,
      hand_landmarks_proto,
      solutions.hands.HAND_CONNECTIONS,
      solutions.drawing_styles.get_default_hand_landmarks_style(),
      solutions.drawing_styles.get_default_hand_connections_style())

    # Get the top left corner of the detected hand's bounding box.
    height, width, _ = annotated_image.shape
    x_coordinates = [landmark.x for landmark in hand_landmarks]
    y_coordinates = [landmark.y for landmark in hand_landmarks]
    text_x = int(min(x_coordinates) * width)
    text_y = int(min(y_coordinates) * height) - MARGIN


    global player
    if(f"{gestures[0].category_name}" == "Closed_Fist"):
      player = GameMove.ROCK

    elif(f"{gestures[0].category_name}" == "Open_Palm"):
      player = GameMove.PAPER
    
    elif(f"{gestures[0].category_name}" == "Victory"):
      player = GameMove.SCISSORS
    
    
  return annotated_image

In [8]:
overlay = cv2.imread('hands.png', cv2.IMREAD_UNCHANGED)

h, w, c = overlay.shape

# Crop the left half of the frame
scissors_image = overlay[:, :w // 3, :]
paper_image = overlay[:, w // 3+10:w // 3*2, :]
rock_image = overlay[:, w // 3*2+10:, :]

# Cámara
vid = cv2.VideoCapture(0)

ret, frame = vid.read()

# Get the width and height of the frame
width = frame.shape[1]
height = frame.shape[0]

# Get the frame rate of the video capture
fps = vid.get(cv2.CAP_PROP_FPS)


countdown = 3
fps_counter = 0
game_ends = False


# Crea objeto FaceLandmarker 
# Obtenr archivo .task y modificar la ruta 
base_options = python.BaseOptions(model_asset_path='C:/Users/danie/Documents/gesture_recognizer.task')
options = vision.GestureRecognizerOptions(base_options=base_options,
                                       num_hands=1,
                                       min_hand_detection_confidence=0.1,
                                       min_tracking_confidence=0.5)
recognizer = vision.GestureRecognizer.create_from_options(options)

while(True):      
    # fotograma a fotograma
    ret, frame = vid.read()
  
    if ret:
        if (fps_counter < fps):
            fps_counter+=1
        elif (fps_counter >= fps):
            countdown-=1
            fps_counter=0


        # Adapta formato
        image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)

        # Detección
        gesture_recognition_result = recognizer.recognize(image)
        
        if (gesture_recognition_result.hand_landmarks != []):

            if (gesture_recognition_result.hand_landmarks[0][9].x < 0.5):

                # Visualiza resultado
                annotated_image = draw_landmarks_on_image(image.numpy_view(), gesture_recognition_result)
                
                cv2.putText(annotated_image, f"{player.name}", (0, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)

                cv2.line(annotated_image, (width//2, 0), (width//2, height//2-70), (0, 0, 0), 6)
                cv2.line(annotated_image, (width//2, height//2), (width//2, height), (0, 0, 0), 6)
                
                if (gesture_recognition_result.hand_landmarks[0][9].x > 0.22 and gesture_recognition_result.hand_landmarks[0][9].x < 0.5 and gesture_recognition_result.hand_landmarks[0][9].y < 0.12):
                   game_ends = False
                   countdown = 3
                   fps_counter = 0
                
                #if (gesture_recognition_result.hand_landmarks[0][9].x < 0.5 and ):

                cv2.putText(annotated_image, "RESTART", (width//2-140, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)

                if (countdown <= 0):
                    cv2.putText(annotated_image, "0", (width//2-31, height//2), cv2.FONT_HERSHEY_SIMPLEX, 3, (255, 255, 255), 8, cv2.LINE_AA)
                else:
                    cv2.putText(annotated_image, f"{countdown}", (width//2-31, height//2), cv2.FONT_HERSHEY_SIMPLEX, 3, (255, 255, 255), 8, cv2.LINE_AA)
                
                if(countdown == 0 and fps_counter == 0):
                    opponent = random.choice(list(GameMove))
                    

                if (opponent.name == "SCISSORS"):
                    annotated_image = cvzone.overlayPNG(annotated_image, scissors_image, [330,160])
                if (opponent.name == "ROCK"):
                    annotated_image = cvzone.overlayPNG(annotated_image, rock_image, [340,160])
                if (opponent.name == "PAPER"):
                    annotated_image = cvzone.overlayPNG(annotated_image, paper_image, [340,160])
                
                
                cv2.putText(annotated_image, f"{opponent.name}", (width-160, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)
                
                
                if(countdown <= 0 and game_ends == False):
                    if (player.name == "ROCK" and opponent.name == "SCISSORS" or
                        player.name == "PAPER" and opponent.name == "ROCK" or
                        player.name == "SCISSORS" and opponent.name == "PAPER" ):
                        end_string = ["YOU WIN!", width//2-70, 255, 255, 255]
                    elif (opponent.name == "ROCK" and player.name == "SCISSORS" or
                        opponent.name == "PAPER" and player.name == "ROCK" or
                        opponent.name == "SCISSORS" and player.name == "PAPER"):
                        end_string = ["YOU LOSE!", width//2-70, 0, 0, 255]
                    elif (opponent.name == "ROCK" and player.name == "ROCK" or
                        opponent.name == "PAPER" and player.name == "PAPER" or
                        opponent.name == "SCISSORS" and player.name == "SCISSORS"):
                        end_string = ["DRAW", width//2-40, 0, 255, 255]
                    
                    game_ends = True

                elif (countdown <= 0):
                    cv2.putText(annotated_image, end_string[0], (end_string[1], height//2+70), cv2.FONT_HERSHEY_SIMPLEX, 1, (end_string[2],end_string[3],end_string[4]), 2, cv2.LINE_AA)
                    

                final_image = annotated_image
                
            else:

                frame[:,:,2] = 255
                cv2.putText(frame, "WARNING!", (0, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                cv2.putText(frame, "HANDS ON LEFT SIDE!", (width//4, height//2), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 4, cv2.LINE_AA)

                final_image = frame

                if (countdown > 0):
                    countdown = 3

        else:
            cv2.putText(image.numpy_view(), f"{player.name}", (0, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)

            cv2.line(image.numpy_view(), (width//2, 0), (width//2, height//2-70), (0, 0, 0), 6)
            cv2.line(image.numpy_view(), (width//2, height//2), (width//2, height), (0, 0, 0), 6)

            cv2.putText(image.numpy_view(), "RESTART", (width//2-140, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)

            if (countdown <= 0):
                cv2.putText(image.numpy_view(), "0", (width//2-31, height//2), cv2.FONT_HERSHEY_SIMPLEX, 3, (255, 255, 255), 8, cv2.LINE_AA)
            else:
                cv2.putText(image.numpy_view(), f"{countdown}", (width//2-31, height//2), cv2.FONT_HERSHEY_SIMPLEX, 3, (255, 255, 255), 8, cv2.LINE_AA)
            
            if(countdown == 0 and fps_counter == 0):
                opponent = random.choice(list(GameMove))

            new_image = np.copy(image.numpy_view())

            if (opponent.name == "SCISSORS"):
                image = cvzone.overlayPNG(new_image, scissors_image, [330,160])
            if (opponent.name == "ROCK"):
                image = cvzone.overlayPNG(new_image, rock_image, [340,160])
            if (opponent.name == "PAPER"):
                image = cvzone.overlayPNG(new_image, paper_image, [340,160])
                

            cv2.putText(image, f"{opponent.name}", (width-160, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)

            if(countdown <= 0 and game_ends == False):
                    if (player.name == "ROCK" and opponent.name == "SCISSORS" or
                        player.name == "PAPER" and opponent.name == "ROCK" or
                        player.name == "SCISSORS" and opponent.name == "PAPER" ):
                        end_string = ["YOU WIN!", width//2-70, 255, 255, 255]
                    elif (opponent.name == "ROCK" and player.name == "SCISSORS" or
                        opponent.name == "PAPER" and player.name == "ROCK" or
                        opponent.name == "SCISSORS" and player.name == "PAPER"):
                        end_string = ["YOU LOSE!", width//2-70, 0, 0, 255]
                    elif (opponent.name == "ROCK" and player.name == "ROCK" or
                        opponent.name == "PAPER" and player.name == "PAPER" or
                        opponent.name == "SCISSORS" and player.name == "SCISSORS"):
                        end_string = ["DRAW", width//2-40, 0, 255, 255]
                    
                    game_ends = True

            elif (countdown <= 0):

                cv2.putText(image, end_string[0], (end_string[1], height//2+70), cv2.FONT_HERSHEY_SIMPLEX, 1, (end_string[2],end_string[3],end_string[4]), 2, cv2.LINE_AA)


            final_image = image
        

        # Muestra fotograma
        cv2.imshow('Vid', final_image)

    
    # Detenemos pulsado ESC
    if cv2.waitKey(20) == 27:
        break
  
# Libera el objeto de captura
vid.release()
# Destruye ventanas
cv2.destroyAllWindows()